In [1]:
import pandas as pd
import os
import zipfile
from pandas import ExcelWriter
import numpy as np

In [2]:
# Please assign the following variables
category_name = ''
workbook_name = '' # excel file name eventually output
year = ['','']

In [3]:
relevant_path = os.getcwd()
ext = 'zip'
file_names = []
for fn in os.listdir(relevant_path):
    if fn.endswith(ext):
        file_names.append(fn)

# unzip the data we download
for file in file_names:   
    with zipfile.ZipFile(file,"r") as zip_ref:
        zip_ref.extractall(relevant_path)

In [4]:
lwd = ''
for wd in os.getcwd().split('\\')[:-1]:
    lwd = lwd + '\\' + wd
lwd = lwd.lstrip('\\')

zcta_state_mapping_file = 'zcta_state_mapping.csv'
zcta_state_mapping = pd.read_csv(lwd + '\\' + zcta_state_mapping_file)

zcta_zip_mapping_file = 'zip_to_zcta_2017.xlsx'
zcta_zip_mapping = pd.read_excel(lwd + '\\' + zcta_zip_mapping_file)

zcta_zip_mapping = zcta_zip_mapping[['ZIP_CODE','ZCTA']]

zcta_zip_mapping = zcta_zip_mapping.rename(columns={'ZCTA':'ZCTA5',
                                'ZIP_CODE':'ZIP'})
zcta_zip_mapping['ZIP'] = zcta_zip_mapping['ZIP'].astype(str).str.zfill(5)
zcta_state_zip_mapping = zcta_state_mapping.merge(zcta_zip_mapping, how='left', on='ZCTA5')

zcta_state_zip_mapping[['ZCTA5','ZIP']] = zcta_state_zip_mapping[['ZCTA5','ZIP']].astype(str)

zcta_state_zip_mapping['ZCTA5'] = zcta_state_zip_mapping['ZCTA5'].str.zfill(5)
zcta_state_zip_mapping['ZIP'] = zcta_state_zip_mapping['ZIP'].str.zfill(5)

In [5]:
# The following code is for selecting the 'estimate' variable then generate a csv file. Use the csv file to update 'Mapping Table
# With Source'.
####################### Code Below ##########################################

# metadata = [] 
# for f in os.listdir(relevant_path):
#     if f.endswith('metadata.csv'):
#         metadata.append(f)
# var_name = pd.read_csv(metadata[0])
# var_name = var_name.iloc[2:]
# var_name = var_name[var_name['GEO.id'].str.contains('EST')]
#var_name.to_csv('var_name.csv',index=False)

In [11]:
var_mapping_file = 'Mapping Table With Source.xlsx'
var_mapping = pd.read_excel(lwd + '\\' + var_mapping_file)

var_mapping = var_mapping[var_mapping['Level 1'] == category_name]
var_mapping = var_mapping[['Variable Code','GEO.id']]

In [12]:
d = {}
d.update(zip(var_mapping['GEO.id'],var_mapping['Variable Code']))

In [13]:
# read file year by year
file_names = []
for fn in os.listdir(relevant_path):
    if fn.endswith('ann.csv'):
        file_names.append(fn)

year_data = []

for f in file_names:
    year_data.append(pd.read_csv(f))

C:\Users\PL_DELL3688_TWO\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2728: DtypeWarning: Columns (1,3,4,5,6,9,10,11,12,15,16,17,18,21,22,23,24,27,28,29,30,33,34,35,36,39,40,41,42,45,46,47,48,51,52,53,54,57,58,59,60,63,64,65,66,69,70,71,72,75,76,77,78,81,82,83,84,87,88,89,90,93,94,95,96,99,100,101,102,105,106,107,108,113,114,117,118,119,120,123,124,125,126,129,130,131,132,135,136,137,138,141,142,143,144,147,148,149,150,153,154,155,156,159,160,161,162,165,166,167,168,173,174,177,178,179,180,183,184,185,186,189,190,191,192,195,196,197,198,201,202,203,204,207,208,209,210,213,214,215,216,219,220,221,222,225,226,227,228,233,234,237,238,239,240,243,244,245,246,249,250,251,252,255,256,257,258,261,262,263,264,267,268,269,270,273,274,275,276,281,282,285,286,287,288,291,292,293,294,297,298,299,300,303,304,305,306,309,310,311,312,315,316,317,318,321,322,323,324,329,330,333,334,335,336,339,340,341,342,345,346,347,348,351,352,353,354,357,358,359,360,363,364,365,366,369,370,371,372,3

In [14]:
def processRawData(df):
    
    # select the columns we need
    for colname in df.columns[3:]:
        if sum(var_mapping['GEO.id'].str.contains(colname)) == 0:
            df = df.drop(colname,axis=1)

    df.drop(['GEO.id','GEO.display-label'], axis=1, inplace=True)
    df = df.rename(columns={'GEO.id2':'ZCTA5'})
    df['ZCTA5'] = df['ZCTA5'].astype(str)
    df = df.merge(zcta_state_zip_mapping, how='left', on='ZCTA5')
    # change column order
    col = df.columns.tolist()
    col = col[-2:] + col[:-2]
    df = df[col]

    df = df.drop('ZCTA5',axis=1)
    df = df.drop(index=df[df.ZIP.isnull()].index[1:],axis=0)
    df.iloc[0,0] = 'State'
    df.iloc[0,1] = 'ZIP'

    # update column names using variable code
    for cols in df.columns[2:]:
        df = df.rename(columns={cols:d[cols]})

    df = df.replace(['-','(X)','+'],np.nan)
    df = df.rename(columns={'State':'',
                                 'ZIP':''})
    return(df)

In [15]:
writer = ExcelWriter(workbook_name)

In [16]:
for df,y in zip(year_data,year):
    data = processRawData(df)
    data.to_excel(writer, y, index=False)

In [17]:
writer.save()